In [ ]:
## set covid19_datasciente as path for python find bulletin package
import sys
from pathlib import Path
from os import getcwd
sys.path.append(str(Path(getcwd()).parent.parent)) ## ../../covid19datascience* <- set parent level here

In [ ]:
import pandas as pd
from epiweeks import Week
from datetime import date, timedelta
from os.path import join

from bulletin import default_input, default_output, root, hoje, data_comeco_pandemia
from bulletin.systems.casos_confirmados import CasosConfirmados
from bulletin.systems.notifica import Notifica

In [ ]:
# casos_confirmados = CasosConfirmados(force=True, hard=True)
# casos_confirmados.shape()
# regionais = pd.read_csv('bulletin/resources/csv/regionais.csv')

In [ ]:
#Usar 0 para hoje e 1 para ontem
# algarismo = 1
# data = datetime.today() - timedelta(algarismo)
# dia = data.day
# mes = data.month
# ano = data.year

# casos = pd.read_csv(f"input/informe_epidemiologico_{str(dia).zfill(2)}_{str(mes).zfill(2)}_{ano}_geral.csv",
#                     sep = ';',
#                     converters = {
#                         'DATA_DIAGNOSTICO' : pd.to_datetime,
#                         'DATA_OBITO' : pd.to_datetime
#                    }
#                    )

# casos = casos.loc[(~(casos['IBGE_RES_PR'] == 9999999))]
# casos['ibge'] = casos['IBGE_RES_PR'].astype('str').apply(lambda row: row[0:6])
# regionais = regionais[['ibge', 'nu_reg']]
# regionais['ibge'] = regionais['ibge'].astype('str')
# casos = casos.merge(how = 'left', on = 'ibge', right = regionais)
# casos.shape
# casos = casos_confirmados.get_casos()

In [ ]:
cc = CasosConfirmados()
cc.load(f"cc_{hoje.strftime('%d_%m_%Y')}", compress=True)
# cc.load(f"cc_{str(date.today().day).zfill(2)}_{str(date.today().month).zfill(2)}_{str(date.today().year).zfill(4)}", compress = True)

In [ ]:
casos = cc.df.loc[((cc.df['uf_residencia'] == 'PR')) & (cc.df['classificacao_final'] == 2)].copy()
# casos.loc[(casos['OBITO'] == 'SIM') & (~(casos['IBGE_RES_PR'] == 9999999))]
del cc

In [ ]:
nf = Notifica()
municipios = nf.tables['municipios']
municipios = municipios[['ibge', 'rs']].rename(columns={'ibge':'ibge_residencia'})
casos = casos.merge(how = 'left', on = 'ibge_residencia', right = municipios)

In [ ]:
obitos = casos.loc[casos['evolucao'] == 2]
obitos.shape
# obitos = casos_confirmados.get_obitos()

In [ ]:
# casos['DATA_DIAGNOSTICO'] = casos['DATA_DIAGNOSTICO'].apply(lambda x: pd.to_datetime(x, format='%d/%m/%Y'))
# obitos['DATA_OBITO'] = obitos['DATA_OBITO'].apply(lambda x: pd.to_datetime(x, format='%d/%m/%Y'))

In [ ]:
casos['se'] = casos['data_diagnostico'].apply(lambda x: Week.fromdate(x))
obitos['se'] = obitos['data_cura_obito'].apply(lambda x: Week.fromdate(x))

In [ ]:
tabela_casos = casos.groupby(['rs','se'])[['rs']].count().unstack()
tabela_casos = tabela_casos.droplevel(0, axis=1)

In [ ]:
tabela_casos.columns = [ x.week for x in tabela_casos.columns.values]

In [ ]:
tabela_obitos = obitos.groupby(['rs','se'])[['rs']].count().unstack()
tabela_obitos = tabela_obitos.droplevel(0, axis=1)

In [ ]:
tabela_obitos.columns = [ x.week for x in tabela_obitos.columns.values]

In [ ]:
total_casos = tabela_casos.sum(0)
tabela_casos = tabela_casos.append(total_casos, ignore_index = True)

In [ ]:
total_obitos = tabela_obitos.sum(0)
tabela_obitos = tabela_obitos.append(total_obitos, ignore_index = True)

In [ ]:
tabela_casos['rs'] = list(range(1, 23)) + ['Paraná']
tabela_casos = tabela_casos.set_index('rs')

In [ ]:
tabela_obitos['rs'] = list(range(1, 23)) + ['Paraná']
tabela_obitos = tabela_obitos.set_index('rs')

In [ ]:
tabela_casos = tabela_casos.fillna(0).astype(int)
tabela_casos

In [ ]:
tabela_obitos = tabela_obitos.fillna(0).astype(int)
tabela_obitos

In [ ]:
tabela_casos.to_excel(join(default_output,'matriz','TABELA_casos_por_se.xlsx'), index = False)
tabela_obitos.to_excel(join(default_output,'matriz','TABELA_obitos_por_se.xlsx'), index = False)